## Graph Neural Networks with PyTorch

Ref: https://docs.dgl.ai/tutorials/blitz/1_introduction.html

In [1]:
# !pip install dgl

import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F
import dgl.data

Using backend: pytorch


In [2]:
dataset = dgl.data.CoraGraphDataset()
g = dataset[0]

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.


In [3]:
print('Number of categories:', dataset.num_classes)
print('Node features')
print(g.ndata)
print('Edge features')
print(g.edata)

Number of categories: 7
Node features
{'feat': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]), 'label': tensor([3, 4, 4,  ..., 3, 3, 3]), 'test_mask': tensor([False, False, False,  ...,  True,  True,  True]), 'val_mask': tensor([False, False, False,  ..., False, False, False]), 'train_mask': tensor([ True,  True,  True,  ..., False, False, False])}
Edge features
{}


In [7]:
print('Node features')
print(g.ndata['feat'].shape)
print('Edge features')
print(g.edata)

Node features
torch.Size([2708, 1433])
Edge features
{}


In [4]:
from dgl.nn import GraphConv
class GCN(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats) #1432, 128
        self.conv2 = GraphConv(h_feats, num_classes) # 128, 7

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

In [9]:
# Create the model with given dimensions

model = GCN(g.ndata['feat'].shape[1], 32, dataset.num_classes)
print(model)

GCN(
  (conv1): GraphConv(in=1433, out=32, normalization=both, activation=None)
  (conv2): GraphConv(in=32, out=7, normalization=both, activation=None)
)


In [10]:
def train(g, model):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    best_val_acc = 0
    best_test_acc = 0

    features = g.ndata['feat']
    labels = g.ndata['label']
    train_mask = g.ndata['train_mask']
    val_mask = g.ndata['val_mask']
    test_mask = g.ndata['test_mask']
    for e in range(100):
        # Forward
        logits = model(g, features) #0 0 0 0.2 0.4 0.9 0.1

        # Compute prediction
        pred = logits.argmax(1)

        # Compute loss
        # Note that you should only compute the losses of the nodes in the training set.
        loss = F.cross_entropy(logits[train_mask], labels[train_mask])

        # Compute accuracy on training/validation/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

        # Save the best validation accuracy and the corresponding test accuracy.
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if e % 5 == 0:
            print('In epoch {}, loss: {:.3f}, val acc: {:.3f} (best {:.3f}), test acc: {:.3f} (best {:.3f})'.format(
                e, loss, val_acc, best_val_acc, test_acc, best_test_acc))


In [11]:
model = GCN(g.ndata['feat'].shape[1], 16, dataset.num_classes)
train(g, model)

In epoch 0, loss: 1.945, val acc: 0.154 (best 0.154), test acc: 0.149 (best 0.149)
In epoch 5, loss: 1.888, val acc: 0.554 (best 0.560), test acc: 0.586 (best 0.584)
In epoch 10, loss: 1.805, val acc: 0.630 (best 0.630), test acc: 0.643 (best 0.643)
In epoch 15, loss: 1.698, val acc: 0.686 (best 0.686), test acc: 0.667 (best 0.667)
In epoch 20, loss: 1.567, val acc: 0.692 (best 0.694), test acc: 0.686 (best 0.680)
In epoch 25, loss: 1.415, val acc: 0.706 (best 0.706), test acc: 0.698 (best 0.698)
In epoch 30, loss: 1.246, val acc: 0.720 (best 0.720), test acc: 0.722 (best 0.722)
In epoch 35, loss: 1.070, val acc: 0.724 (best 0.726), test acc: 0.725 (best 0.726)
In epoch 40, loss: 0.896, val acc: 0.742 (best 0.742), test acc: 0.738 (best 0.738)
In epoch 45, loss: 0.734, val acc: 0.758 (best 0.758), test acc: 0.753 (best 0.753)
In epoch 50, loss: 0.592, val acc: 0.768 (best 0.768), test acc: 0.767 (best 0.764)
In epoch 55, loss: 0.474, val acc: 0.774 (best 0.774), test acc: 0.774 (best 0